In [7]:
import os
import sys
from datetime import date
import hls4ml
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt

#from utils import plotting

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
sys.path.append("/mnt/storage1/akshaymalige/demo_new/rt_detect_le_events_dune-main/src")
os.environ['PATH'] = '/scratch/am_vitis/Vitis/2023.1/bin:'+os.environ['PATH']

from dune.model import model_t  # noqa: E402
from dune.util import common_util, marley_dataset_util
from hls4ml.model import profiling
from dune.model import model_t, predict_t  # noqa: E402
from dune.fpga.hls_util import hls_util  # noqa: E402

model = model_t()
model.load_weights()

input_dim: (64, 480, 1)
cropped_dim: (32, 64, 1)
sum_extra_padding: ((0, 0), (16, 16), (0, 0))
sum_kernel_size: (2, 8, 1)
loading weights from: /mnt/storage1/akshaymalige/demo_new/rt_detect_le_events_dune-main/out/classifier_2d_s544_o2_i64_480_1_c32_64_1_q/le/model.003-026-0.02916.h5


In [8]:
hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(
    layers=['Activation'],
    rounding_mode='AP_RND_CONV',
    saturation_mode='AP_SAT')
if model.name.startswith('classifier'):
    reuse_factor = 1
    default_ib = 9
    default_fb = 18
    io_type = "io_stream"
default_dtype = f'ap_fixed<{default_ib+default_fb}, {default_ib}>'
pool_dtype = default_dtype
in_layer_dtype =  pool_dtype
out_layer_dtype = default_dtype
hls_config = hls4ml.utils.config_from_keras_model(model.tf_model,
                                                      granularity='name',
                                                      default_precision=default_dtype,
                                                      default_reuse_factor=reuse_factor)
hls_config['Model']['Strategy'] = 'Resource'
if model.name.startswith('classifier'):
    hls_config['LayerName']['input_1']['Precision'] = in_layer_dtype
    hls_config['LayerName']['output_linear']['Precision'] = out_layer_dtype
    if 'q_conv2d_2' in hls_config['LayerName'].keys():
        hls_config['LayerName']['q_conv2d_2']['ReuseFactor'] = 128
        hls_config['LayerName']['q_dense']['ReuseFactor'] = 128
        in_local_id = 18
    else:
        in_local_id = 15

print("-----------------------------------")
print(hls_config)
print("-----------------------------------")
print(model.name)

proj_dir = os.path.join(model.out_dir, 'hls4ml_prj_u250_axi')
hls_model = hls4ml.converters.convert_from_keras_model(model.tf_model,
                                                           hls_config=hls_config,
                                                           output_dir=proj_dir,
                                                           backend='VivadoAccelerator',
                                                           board='alveo-u250',
                                                           part='xcu250-figd2104-2L-e',
                                                           io_type=io_type,
                                                       driver='python',
                                                       interface='axi_stream'
                                                           )
model.hls_model = hls_model
backend = hls4ml.backends.get_backend('VivadoAccelerator')

Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 64, 480, 1]], output shape: [None, 64, 480, 1]
Layer name: zero_padding2d, layer type: ZeroPadding2D, input shapes: [[None, 64, 480, 1]], output shape: [None, 64, 512, 1]
Layer name: sum_pooling2d, layer type: AveragePooling2D, input shapes: [[None, 64, 512, 1]], output shape: [None, 32, 64, 1]
Layer name: q_conv2d, layer type: QConv2D, input shapes: [[None, 32, 64, 1]], output shape: [None, 32, 64, 8]
Layer name: max_pooling2d, layer type: MaxPooling2D, input shapes: [[None, 32, 64, 8]], output shape: [None, 16, 16, 8]
Layer name: q_conv2d_1, layer type: QConv2D, input shapes: [[None, 16, 16, 8]], output shape: [None, 16, 16, 16]
Layer name: max_pooling2d_1, layer type: MaxPooling2D, input shapes: [[None, 16, 16, 16]], output shape: [None, 4, 4, 16]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 4, 4, 16]], output shape: [None, 256]
Layer name: q_dense, layer type: QDense, 

In [9]:
pathlib.Path(proj_dir).mkdir(parents=True, exist_ok=True)
file_path = os.path.join(proj_dir, 'hls_graph.png')

hls4ml.utils.plot_model(model.hls_model,
                            show_shapes=True,
                            show_precision=True,
                            to_file=file_path)
model.hls_model.compile()

Writing HLS project


/mnt/storage1/akshaymalige/demo_new/rt_detect_le_events_dune-main/.python/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
[WARNING] Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
/mnt/storage1/akshaymalige/demo_new/rt_detect_le_events_dune-main/.python/lib/python3.10/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


Done


In [11]:
hu = hls_util(model)
dataset = hu.default_dataset(length=4096)
X_hls, X_py, _ = hu.load_dataset(dataset, exclude_all_zero=False, exclude_non_zero=False)
print("# sample:", X_hls.shape[0])

y_sim_out, sim_trace = model.hls_model.trace(X_hls)
y_out=hls_model.predict(X_hls)
print(y_out)

print("writing:", os.path.join(proj_dir, "tb_data", "tb_input_features_x.dat"))
with open(os.path.join(proj_dir, "tb_data", "tb_input_features_x.dat"), 'w') as f:
    for i in range(len(X_hls)):
        f.write(", ".join([str(x) for x in X_hls[i].flatten()]) + '\n')

print("writing:", os.path.join(proj_dir, "tb_data", "tb_output_predictions_x.dat"))
with open(os.path.join(proj_dir, "tb_data", "tb_output_predictions_x.dat"), 'w') as f:
    for i in range(len(y_out)):
        f.write(" ".join([str(y) for y in y_out[i].flatten()]) + '\n')

injecting 2 all-zero NB samples every batch
per_batch_sizes: [2, 2, 2, 2]
data_dir: /scratch/jsc2268/slices_64_portable
file_paths: [['/scratch/jsc2268/slices_64_portable/APA_nb_bb.h5'], ['/scratch/jsc2268/slices_64_portable/APA_es_bb.h5'], ['/scratch/jsc2268/slices_64_portable/APA_cc_bb.h5']]
file_sizes: [[32768], [32768], [32768]]
max_boxes: 7
# sample: 4096
Recompiling myproject with tracing
Writing HLS project


[WARNING] Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


Done
[[  3.9431152  -3.9927368]
 [  2.0073853  -2.0291748]
 [-20.65448    20.955688 ]
 ...
 [ -3.3521729   3.4040527]
 [  4.607422   -4.66803  ]
 [  4.607422   -4.66803  ]]
writing: /mnt/storage1/akshaymalige/demo_new/rt_detect_le_events_dune-main/out/classifier_2d_s544_o2_i64_480_1_c32_64_1_q/le/hls4ml_prj_u250_axi/tb_data/tb_input_features_x.dat
writing: /mnt/storage1/akshaymalige/demo_new/rt_detect_le_events_dune-main/out/classifier_2d_s544_o2_i64_480_1_c32_64_1_q/le/hls4ml_prj_u250_axi/tb_data/tb_output_predictions_x.dat


In [6]:
hls_model.build(csim=False,synth=True,export=True)
hls4ml.report.read_vivado_report(proj_dir)


****** Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2023.1 (64-bit)
  **** SW Build 3854077 on May  4 2023
  **** IP Build 3864474 on Sun May  7 20:36:21 MDT 2023
  **** SharedData Build 3865790 on Sun May 07 13:33:03 MDT 2023
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyright 2022-2023 Advanced Micro Devices, Inc. All Rights Reserved.

source /scratch/am_vitis/Vitis_HLS/2023.1/scripts/vitis_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/scratch/am_vitis/Vitis_HLS/2023.1/bin/unwrapped/lnx64.o/vitis_hls'
INFO: [HLS 200-10] For user 'akshaymalige' on host 'cleopatra.nevis.columbia.edu' (Linux_x86_64 version 3.10.0-1160.31.1.el7.x86_64) on Mon Oct 30 14:53:34 EDT 2023
INFO: [HLS 200-10] On os "CentOS Linux release 7.9.2009 (Core)"
INFO: [HLS 200-10] In directory '/mnt/storage1/akshaymalige/demo_new/rt_detect_le_events_dune-main/out/classifier_2d_s544_o2_i64_480_1_c32_64_1_q/le/hls4ml_prj_u250'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-15

In [9]:
platform = 'xilinx_u250_gen3x16_xdma_4_1_202210_1'
project_name=hls_model.config.get_project_name()
abs_path_dir = os.path.abspath(hls_model.config.get_output_dir())
ip_repo_path = abs_path_dir + '/' + project_name + '_prj' + '/solution1/impl/ip'
os.chdir(abs_path_dir)
os.makedirs('xo_files', exist_ok=True)
# os.system('vivado -mode batch -source design.tcl')
os.system('vivado -mode batch -source build_prj.tcl')
# os.makedirs('xclbin_files', exist_ok=True)
# os.chdir(abs_path_dir + '/xclbin_files')
# vitis_cmd = (
#             "v++ -t hw --platform "
#             + platform
#             + " --link ../xo_files/"
#             + project_name
#             + "_kernel.xo -o'"
#             + project_name
#             + "_kernel.xclbin' --user_ip_repo_paths "
#             + ip_repo_path
#         )
# os.system(vitis_cmd)


****** Vivado v2023.1 (64-bit)
  **** SW Build 3865809 on Sun May  7 15:04:56 MDT 2023
  **** IP Build 3864474 on Sun May  7 20:36:21 MDT 2023
  **** SharedData Build 3865790 on Sun May 07 13:33:03 MDT 2023
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyright 2022-2023 Advanced Micro Devices, Inc. All Rights Reserved.

source build_prj.tcl
# array set opt {
#     reset      0
#     csim       1
#     synth      1
#     cosim      1
#     validation 1
#     export     0
#     vsynth     0
#     fifo_opt   0
# }
# set tcldir [file dirname [info script]]
# source [file join $tcldir project.tcl]
## variable project_name
## set project_name "myproject"
## variable backend
## set backend "vivado"
## variable part
## set part "xcu250-figd2104-2L-e"
## variable clock_period
## set clock_period 5
## variable clock_uncertainty
## set clock_uncertainty 12.5%
## variable version
## set version "1.0.0"
# proc remove_recursive_log_wave {} {
#     set tcldir [file dirname [i

ERROR: [Coretcl 2-155] Invalid project file name 'myproject_prj'. File must have a valid Vivado project extension (.xpr/.ppr).


256